## OpenAI Agent SDK


Documentation: https://openai.github.io/openai-agents-python/agents/


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Access your API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
   raise ValueError("OPENAI_API_KEY not found in .env file")

### Basics

In [19]:
## Sync run 
agent = Agent(name="Assistant", instructions="You are a helpful assistant",model="gpt-4o-mini")
result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

## async run 
agent = Agent(
   name="Test Agent",
   instructions="You are a helpful assistant that provides concise responses.",
   model="gpt-4o-mini"
)
result = await Runner.run(agent, "Hello! Are you working correctly?")
print(result.final_output)


Code within itself,  
A loop of endless echoes—  
Infinite unfolds.
Hello! Yes, I'm working correctly. How can I assist you today?


### Some Agent Params

### Best practices for writing instructions
- Be specific: Clearly define the agent’s role, personality, and limitations
- Set boundaries: Explicitly state what topics or actions the agent should avoid
- Define interaction patterns: Explain how the agent should handle various types of inputs
- Establish knowledge boundaries: Clarify what the agent should know and when it should acknowledge uncertainty

In [ ]:
from agents import ModelSettings

In [22]:
advanced_agent = Agent(
   name="Advanced Assistant",
   instructions="""You are a professional, concise assistant who always provides
   accurate information. When you don't know something, clearly state that.
   Focus on giving actionable insights when answering questions.""",
   model="gpt-4o-mini",
   model_settings=ModelSettings(
       temperature=0.3,  # Lower for more deterministic outputs (0.0-2.0)
       max_tokens=1024,  # Maximum length of response
   ),
   tools=[] 
)

### Agent tools use

In [23]:
from agents import WebSearchTool

In [32]:
# Create a research assistant with web search capability
research_assistant = Agent(
   name="Research Assistant",
   instructions="""You are a research assistant that helps users find and summarize information.
   When asked about a topic:
   1. Search the web for relevant, up-to-date information
   2. Synthesize the information into a clear, concise summary
   3. Structure your response with headings and bullet points when appropriate
   4. Always cite your sources at the end of your response
  
   If the information might be time-sensitive or rapidly changing, mention when the search was performed.
   """,
   tools=[WebSearchTool(
      user_location={"type": "approximate", "city": "New York"},  # Provides geographic context for local search queries
      search_context_size='medium'   ## high medium low
      )]
   )

In [33]:
# Usage example (in Jupyter notebook)
summary = Runner.run_sync(research_assistant, "Latest developments in personal productivity apps.")
print(summary)

RunResult:
- Last agent: Agent(name="Research Assistant", ...)
- Final output (str):
    As of March 25, 2025, personal productivity applications have seen significant advancements, particularly through the integration of artificial intelligence (AI) and the development of autonomous AI agents.
    
    **Integration of AI in Productivity Tools**
    
    AI has been increasingly embedded into productivity applications to enhance efficiency and user experience. Notable developments include:
    
    - **AI-Powered Assistants**: Applications like Notion have introduced AI assistants capable of drafting, summarizing, and refining content within user notes, facilitating collaborative projects and individual brainstorming. ([eti.co.ke](https://eti.co.ke/how-ai-is-changing-personal-productivity-apps-you-need/?utm_source=openai))
    
    - **Intelligent Task Management**: Tools such as Todoist have integrated AI to help users prioritize tasks based on deadlines, importance, and workload, su

#### Basic Function tool 

In [2]:
from agents import function_tool

In [3]:
@function_tool
def get_weather(city: str) -> str:
    """
    Get the weather in a city
    """
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-4o-mini",
    tools=[get_weather],
)
results = Runner.run_sync(agent, "what is the weather in new york?")
print(results.final_output)

Sunshine fills the sky,  
New York basks in golden light,  
Joy in every breath.


In [4]:
## comments for the fucntion will be the description of the tool;
get_weather

FunctionTool(name='get_weather', description='Get the weather in a city', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x76506517b1c0>, strict_json_schema=True)

#### Structured output

In [5]:
from pydantic import BaseModel
from typing import List, Optional
# Define person data model
class Person(BaseModel):
   name: str
   role: Optional[str]
   contact: Optional[str]
# Define meeting data model
class Meeting(BaseModel):
   date: str
   time: str
   location: Optional[str]
   duration: Optional[str]
# Define task data model
class Task(BaseModel):
   description: str
   assignee: Optional[str]
   deadline: Optional[str]
   priority: Optional[str]
# Define the complete email data model
class EmailData(BaseModel):
   subject: str
   sender: Person
   recipients: List[Person]
   main_points: List[str]
   meetings: List[Meeting]
   tasks: List[Task]
   next_steps: Optional[str]

In [6]:
# Create an email extraction agent with structured output
email_extractor = Agent(
   name="Email Extractor",
   instructions="""You are an assistant that extracts structured information from emails.
  
   When given an email, carefully identify:
   - Subject and main points
   - People mentioned (names, roles, contact info)
   - Meetings (dates, times, locations)
   - Tasks or action items (with assignees and deadlines)
   - Next steps or follow-ups
  
   Extract this information as structured data. If something is unclear or not mentioned,
   leave those fields empty rather than making assumptions.
   """,
   output_type=EmailData,  # This tells the agent to return data in EmailData format
)

In [7]:
# For simple lists
agent_with_list_output = Agent(
   name="List Generator",
   instructions="Generate lists of items based on the user's request.",
   output_type=list[str],  # Returns a list of strings
)

# For dictionaries
agent_with_dict_output = Agent(
   name="Dictionary Generator",
   instructions="Create key-value pairs based on the input.",
   output_type=dict[
       str, int
   ],  # Returns a dictionary with string keys and integer values
)

# For simple primitive types
agent_with_bool_output = Agent(
   name="Decision Maker",
   instructions="Answer yes/no questions with True or False.",
   output_type=bool,  # Returns a boolean
)

- see context and output types here : https://openai.github.io/openai-agents-python/agents/

#### Agent as tool 

In [9]:
# Specialist agents
note_taking_agent = Agent(
   name="Note Manager",
   instructions="You help users take and organize notes efficiently.",
   # In a real application, this agent would have note-taking tools
)
task_management_agent = Agent(
   name="Task Manager",
   instructions="You help users manage tasks, deadlines, and priorities.",
   # In a real application, this agent would have task management tools
)
# Coordinator agent that uses specialists as tools
productivity_assistant = Agent(
   name="Productivity Assistant",
   instructions="""You are a productivity assistant that helps users organize their work and personal life.
  
   For note-taking questions or requests, use the note_taking tool.
   For task and deadline management, use the task_management tool.
  
   Help the user decide which tool is appropriate based on their request,
   and coordinate between different aspects of productivity.
   """,
   tools=[
       note_taking_agent.as_tool(
           tool_name="note_taking",
           tool_description="For taking, organizing, and retrieving notes and information"
       ),
       task_management_agent.as_tool(
           tool_name="task_management",
           tool_description="For managing tasks, setting deadlines, and tracking priorities"
       )
   ]
)
result = Runner.run_sync(productivity_assistant, "I need to keep track of my project deadlines")
print(result.final_output)

Do you want assistance setting up deadlines in a particular tool or platform?


### Basic handoff

In [10]:
billing_agent = Agent(
   name="Billing Agent",
   instructions="""You are a billing specialist who helps customers with payment issues.
   Focus on resolving billing inquiries, subscription changes, and refund requests.
   If asked about technical problems or account settings, explain that you specialize
   in billing and payment matters only.""",
)

technical_agent = Agent(
   name="Technical Agent",
   instructions="""You are a technical support specialist who helps with product issues.
   Assist users with troubleshooting, error messages, and how-to questions.
   Focus on resolving technical problems only.""",
)

# Create a triage agent that can hand off to specialists
triage_agent = Agent(
   name="Customer Service",
   instructions="""You are the initial customer service contact who helps direct
   customers to the right specialist.
  
   If the customer has billing or payment questions, hand off to the Billing Agent.
   If the customer has technical problems or how-to questions, hand off to the Technical Agent.
   For general inquiries or questions about products, you can answer directly.
  
   Always be polite and helpful, and ensure a smooth transition when handing off to specialists.""",
   handoffs=[billing_agent, technical_agent],  # Direct handoff to specialist agents; let the llm to decide which agent to handoff to
)

- Run process

In [11]:
async def handle_customer_request(request):
   runner = Runner()
   result = await runner.run(triage_agent, request)
   return result


# Example customer inquiries
billing_inquiry = (
   "I was charged twice for my subscription last month. Can I get a refund?"
)
technical_inquiry = (
   "The app keeps crashing when I try to upload photos. How can I fix this? Give me the shortest solution possible."
)
general_inquiry = "What are your business hours?"

# Process the different types of inquiries
billing_response = await handle_customer_request(billing_inquiry)
print(f"Billing inquiry response:\n{billing_response.final_output}\n")

technical_response = await handle_customer_request(technical_inquiry)
print(f"Technical inquiry response:\n{technical_response.final_output}\n")

general_response = await handle_customer_request(general_inquiry)
print(f"General inquiry response:\n{general_response.final_output}")

Billing inquiry response:
I'm sorry to hear about the double charge. Let's get that sorted out for you. Can you provide me with your account details or the transaction IDs for those charges? This will help me process your refund more efficiently.

Technical inquiry response:
Try these steps: 

1. Clear app cache.
2. Restart the app.
3. Check for updates.
4. Reboot your device.

This should help with the crashing issue.

General inquiry response:
Our business hours are Monday to Friday, 9:00 AM to 5:00 PM. If you have any specific questions, feel free to ask!


### Customizing handoffs
For more control over handoffs, you can use the handoff() function instead of passing agents directly to the handoffs parameter:

In [13]:
from agents import RunContextWrapper
from datetime import datetime

# Create an agent that handles account-related questions
account_agent = Agent(
   name="Account Management",
   instructions="""You help customers with account-related issues such as
   password resets, account settings, and profile updates.""",
)


# Custom handoff callback function
async def log_account_handoff(ctx: RunContextWrapper[None]):
   print(
       f"[LOG] Account handoff triggered at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
   )
   # In a real app, you might log to a database or alert a human supervisor


# Create a triage agent with customized handoffs
enhanced_triage_agent = Agent(
   name="Enhanced Customer Service",
   instructions="""You are the initial customer service contact who directs
   customers to the right specialist.
  
   If the customer has billing or payment questions, hand off to the Billing Agent.
   If the customer has technical problems, hand off to the Technical Agent.
   If the customer needs to change account settings, hand off to the Account Management agent.
   For general inquiries, you can answer directly.""",
   handoffs=[
       billing_agent,  # Basic handoff
       handoff(  # Customized handoff
           agent=account_agent,
           on_handoff=log_account_handoff,  # Callback function
           tool_name_override="escalate_to_account_team",  # Custom tool name
           tool_description_override="Transfer the customer to the account management team for help with account settings, password resets, etc.",
       ),
       technical_agent,  # Basic handoff
   ],
)

result = await Runner.run(
   enhanced_triage_agent, "I need to change my password."
)

[LOG] Account handoff triggered at 2025-03-31 04:23:27


### Passing data during handoffs 
With this setup, when the service agent decides to hand off to the escalation agent, it will provide structured data about why the escalation is happening. The system will validate this data against the EscalationData model before passing it to the process_escalation callback.

In [14]:
from pydantic import BaseModel
from typing import Optional
from agents import Agent, handoff, RunContextWrapper

# Define the data structure to pass during handoff
class EscalationData(BaseModel):
   reason: str
   priority: Optional[str]
   customer_tier: Optional[str]


# Handoff callback that processes the escalation data
async def process_escalation(ctx: RunContextWrapper, input_data: EscalationData):
   print(f"[ESCALATION] Reason: {input_data.reason}")
   print(f"[ESCALATION] Priority: {input_data.priority}")
   print(f"[ESCALATION] Customer tier: {input_data.customer_tier}")
   # You might use this data to prioritize responses, alert human agents, etc.
# Create an escalation agent
escalation_agent = Agent(
   name="Escalation Agent",
   instructions="""You handle complex or sensitive customer issues that require
   special attention. Always address the customer's concerns with extra care and detail.""",
)

# Create a service agent that can escalate with context
service_agent = Agent(
   name="Service Agent",
   instructions="""You are a customer service agent who handles general inquiries.
  
   For complex issues, escalate to the Escalation Agent and provide:
   - The reason for escalation
   - Priority level (Low, Normal, High, Urgent)
   - Customer tier if mentioned (Standard, Premium, VIP)""",
   handoffs=[
       handoff(
           agent=escalation_agent,
           on_handoff=process_escalation,
           input_type=EscalationData,
       )
   ],
)

In [15]:
from agents import trace

### Trace and conversation history

In [21]:
with trace("customer service workflow"): 
    first_result = await Runner.run(service_agent, f"How are you, i got a serious problem with my account")
    second_result = await Runner.run(service_agent, first_result.to_input_list()+[{"role": "user", "content": "I want to talk to your manager, please escalate, it is high priority. I am VIP customer"}])
    ## passing the conversation history to the agent from previous results 
    print(f"1st response: {first_result.final_output}")
    print(f"2nd response: {second_result.final_output}")

[ESCALATION] Reason: Customer requests escalation to speak with a manager about a serious account issue.
[ESCALATION] Priority: High
[ESCALATION] Customer tier: VIP
1st response: I'm sorry to hear you're having an issue with your account. Can you please provide more details about the problem?
2nd response: I've escalated your request to a manager. They will be in touch with you shortly to address your concerns. If there's anything else I can assist you with in the meantime, please let me know.


In [22]:
second_result.to_input_list()

[{'content': 'How are you, i got a serious problem with my account',
  'role': 'user'},
 {'id': 'msg_67ea1fef0a588192af7ca9729815d68904aaba7f46c4af9f',
  'content': [{'annotations': [],
    'text': "I'm sorry to hear you're having an issue with your account. Can you please provide more details about the problem?",
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user',
  'content': 'I want to talk to your manager, please escalate, it is high priority. I am VIP customer'},
 {'arguments': '{"reason":"Customer requests escalation to speak with a manager about a serious account issue.","priority":"High","customer_tier":"VIP"}',
  'call_id': 'call_QMnaCT3Qs8S2fLSnS3Ff7Yzi',
  'name': 'transfer_to_escalation_agent',
  'type': 'function_call',
  'id': 'fc_67ea1ff016ac8192b5979e7b0a8e33ea04aaba7f46c4af9f',
  'status': 'completed'},
 {'call_id': 'call_QMnaCT3Qs8S2fLSnS3Ff7Yzi',
  'output': "{'assistant': 'Escalation Agent'}",
  'type'